# Compute Multilinear regression  - Water and Energy limited 

In [1]:
import os
import glob
import netCDF4 as nc

import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
import cartopy as cart
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import seaborn as sns

import numpy as np
import xarray as xr
import cftime
import pandas as pd

# Regridding
# import xesmf as xe


In [2]:
data_path = "/home/m/m301093/data/clean/"

# Functions

In [3]:
def xr_fix_time(xr_in, date_start, date_end):
    
    if xr_in['time'].dt.calendar == 'noleap' or xr_in['time'].dt.calendar == '360_day':
        xr_in = xr_in.convert_calendar(calendar = 'gregorian', align_on = 'date')
        
    else: None
    
    time = pd.date_range(start=date_start, end=date_end, freq='M').to_numpy(dtype='datetime64[ns]')
    xr_out = xr_in.assign_coords(time = time)
    
    return xr_out

In [4]:
def xr_clean(xr_in, dims):
    data = xr_in.copy()
    for i,d in enumerate(dims):
        if d in data.coords and d in data.dims:
            data = data.drop(d)
            data = data.isel({d : 0})
        if d in data.coords:
            data = data.drop(d)
        if d in data.dims and d not in data.coords:
            data = data.isel({d : 0})
        if data.attrs == {}:
            None
        else:
            if d in data.data_vars:
                data = data.drop_vars(d)
    xr_out = data
    return xr_out

In [5]:
def lon180(ds):
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds = ds.sortby(ds.lon)
    return ds

In [6]:
# import xesmf as xe

# def xr_regrid(data, method, lon_bnds, lat_bnds, xres, yres):
#     # Regrid XARRAY using xESMF library (https://xesmf.readthedocs.io/en/stable/index.html)
#     # data: input data
#     # method: "bilinear", "conservative", "conservative_normed", "patch", "nearest_s2d", "nearest_d2s"
    
#     lonmin = lon_bnds[0]; lonmax = lon_bnds[1]
#     latmin = lat_bnds[0]; latmax = lat_bnds[1]
#     xr_out = xe.util.cf_grid_2d(lonmin, lonmax, xres, latmin, latmax, yres)
   
#     regrid = xe.Regridder(data, xr_out, method)

#     data_regrid = regrid(data, keep_attrs=True)

#     return data_regrid

In [7]:
def xr_multimodel_sign(xr_in, models):
    # Computes the Multimodel MEAN and STD
    # xr_in: list of xarray models
    mm = xr.concat(xr_in, dim = "esm", coords = "minimal", compat = "override")
    
    std = xr.concat(xr_in, dim = "esm", coords = "minimal", compat = "override").std(dim = "esm")
    
    # Multimodel agreement in the sign of the sensitivity
    sign = xr.where(mm > 0, 1, -1)
    agreement = (np.abs(sign.sum(dim = "esm")/len(models)))
    
    mean = mm.mean(dim = "esm")
    return (mean, std, agreement)

In [8]:
def cell_weight(ds):
    R = 6.371e6
    dϕ = np.deg2rad(ds.lat[1] - ds.lat[0])
    dλ = np.deg2rad(ds.lon[1] - ds.lon[0])
    dlat = R * dϕ * xr.ones_like(ds.lon)
    dlon = R * dλ * np.cos(np.deg2rad(ds.lat))
    cell_area = dlon * dlat
    return(cell_area)

In [9]:
#### --------- Mann-Whitney Test --------- ####
from scipy.stats import mannwhitneyu

# Test applied on a grid-cell basis. For every gid-cell, the statistical difference between two time series is computed.
# 1st time series: Nino years' (DS_models_hist_nino)
# 2nd time series: Reference climatology (DS_models_hist_clim or DS_models_hist_clim_neutral)  

## -- Function for grid-cell operations -- ##
def multi_apply_along_axis(func1d, axis, arrs, *args, **kwargs):
    """
    Given a function `func1d(A, B, C, ..., *args, **kwargs)`  that acts on 
    multiple one dimensional arrays, apply that function to the N-dimensional
    arrays listed by `arrs` along axis `axis`
    
    If `arrs` are one dimensional this is equivalent to::
    
        func1d(*arrs, *args, **kwargs)
    
    If there is only one array in `arrs` this is equivalent to::
    
        numpy.apply_along_axis(func1d, axis, arrs[0], *args, **kwargs)
        
    All arrays in `arrs` must have compatible dimensions to be able to run
    `numpy.concatenate(arrs, axis)`
    
    Arguments:
        func1d:   Function that operates on `len(arrs)` 1 dimensional arrays,
                  with signature `f(*arrs, *args, **kwargs)`
        axis:     Axis of all `arrs` to apply the function along
        arrs:     Iterable of numpy arrays
        *args:    Passed to func1d after array arguments
        **kwargs: Passed to func1d as keyword arguments
    """
    # Concatenate the input arrays along the calculation axis to make one big
    # array that can be passed in to `apply_along_axis`
    carrs = np.concatenate(arrs, axis)
    
    # We'll need to split the concatenated arrays up before we apply `func1d`,
    # here's the offsets to split them back into the originals
    offsets=[]
    start=0
    for i in range(len(arrs)-1):
        start += arrs[i].shape[axis]
        offsets.append(start)
            
    # The helper closure splits up the concatenated array back into the components of `arrs`
    # and then runs `func1d` on them
    def helperfunc(a, *args, **kwargs):
        arrs = np.split(a, offsets)
        return func1d(*[*arrs, *args], **kwargs)
    
    # Run `apply_along_axis` along the concatenated array
    return np.apply_along_axis(helperfunc, axis, carrs, *args, **kwargs)


def xr_multipletest(p, alpha=0.05, method='fdr_bh', **multipletests_kwargs):
    """Apply statsmodels.stats.multitest.multipletests for multi-dimensional xr.objects."""
    from statsmodels.stats.multitest import multipletests
    # stack all to 1d array
    p_stacked = p.stack(s=p.dims)
    # mask only where not nan: https://github.com/statsmodels/statsmodels/issues/2899
    mask = np.isfinite(p_stacked)
    pvals_corrected = np.full(p_stacked.shape, np.nan)
    reject = np.full(p_stacked.shape, np.nan)
    # apply test where mask
    reject[mask] = multipletests(
        p_stacked[mask], alpha=alpha, method=method, **multipletests_kwargs)[0]
    pvals_corrected[mask] = multipletests(
        p_stacked[mask], alpha=alpha, method=method, **multipletests_kwargs)[1]

    def unstack(reject, p_stacked):
        """Exchange values from p_stacked with reject (1d array) and unstack."""
        xreject = p_stacked.copy()
        xreject.values = reject
        xreject = xreject.unstack()
        return xreject

    reject = unstack(reject, p_stacked)
    pvals_corrected = unstack(pvals_corrected, p_stacked)
    return reject, pvals_corrected


In [10]:
def lag_linregress_3D(x, y, lagx=0, lagy=0):
    """
    Input: Two xr.Datarrays of any dimensions with the first dim being time. 
    Thus the input data could be a 1D time series, or for example, have three dimensions (time,lat,lon). 
    Datasets can be provied in any order, but note that the regression slope and intercept will be calculated
    for y with respect to x.
    Output: Covariance, correlation, regression slope and intercept, p-value, and standard error on regression
    between the two datasets along their aligned time dimension.  
    Lag values can be assigned to either of the data, with lagx shifting x, and lagy shifting y, with the specified lag amount. 
    """ 
    #1. Ensure that the data are properly alinged to each other. 
    x,y = xr.align(x,y)
    
    #2. Add lag information if any, and shift the data accordingly
    if lagx!=0:
        #If x lags y by 1, x must be shifted 1 step backwards. 
        #But as the 'zero-th' value is nonexistant, xr assigns it as invalid (nan). Hence it needs to be dropped
        x   = x.shift(time = -lagx).dropna(dim='time')
        #Next important step is to re-align the two datasets so that y adjusts to the changed coordinates of x
        x,y = xr.align(x,y)

    if lagy!=0:
        y   = y.shift(time = -lagy).dropna(dim='time')
        x,y = xr.align(x,y)
 
    #3. Compute data length, mean and standard deviation along time axis for further use: 
    n     = x.shape[0]
    xmean = x.mean(axis=0)
    ymean = y.mean(axis=0)
    xstd  = x.std(axis=0)
    ystd  = y.std(axis=0)
    
    #4. Compute covariance along time axis
    cov   =  np.sum((x - xmean)*(y - ymean), axis=0)/(n)
    
    #5. Compute correlation along time axis
    cor   = cov/(xstd*ystd)
    
    #6. Compute regression slope and intercept:
    slope     = cov/(xstd**2)
    intercept = ymean - xmean*slope
    y_pred =  intercept + slope*x
    res = y - y_pred

    #7. Compute P-value and standard error
    #Compute t-statistics
    tstats = cor*np.sqrt(n-2)/np.sqrt(1-cor**2)
    stderr = slope/tstats
    
    from scipy.stats import t
    pval   = t.sf(tstats, n-2)*2
    pval   = xr.DataArray(pval, dims=cor.dims, coords=cor.coords)

    #return cov,cor,slope,intercept,pval,stderr
    results = {}
    results["cor"] = cor
    results["coef"] = slope
    results["pval"] = pval
    return results

## Import preprocessed data

In [11]:
filepath = glob.glob(os.path.join('/home/m/m301093/data'  +'/'+ "era5_lai_196001-198912_mean.nc"))[0]
xr_lai_era5 = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"])
xr_lai_era5 = xr_lai_era5.isel(time = 0).lai_hv
xr_lai_era5 = xr_lai_era5.rename({"latitude": "lat", "longitude": "lon"})
xr_lai_era5 = lon180(xr_lai_era5)
xr_lai_era5 = xr_lai_era5.sel(lat = slice(90,-60))

In [12]:
models = [ "ACCESS-ESM1-5", "CanESM5", "IPSL-CM6A-LR", "MPI-ESM1-2-LR", "UKESM1-0-LL"]

In [13]:
scenario = 'ssp126Lu'
xr_aff = []
for i,mm in enumerate(models):
    filepath = glob.glob(os.path.join(data_path + 'xr_' + scenario + "_" + mm + '.nc'))[0]                                       ## List of files sorted by name
    content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"],engine = 'netcdf4',chunks={"time": 240})
    xr_aff.append(content)


scenario = 'ssp370'
xr_ctl = []
for i,mm in enumerate(models):
    filepath = glob.glob(os.path.join(data_path + 'xr_' + scenario + "_" + mm + '.nc'))[0]                                       ## List of files sorted by name
    content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"],engine = 'netcdf4',chunks={"time": 240})
    xr_ctl.append(content)


In [14]:
scenario = 'ssp126Lu'
xr_aff_pft = []
for i,mm in enumerate(models):
    filepath = glob.glob(os.path.join(data_path + 'xr_' + scenario + "_" + mm + '_pft.nc'))[0]                                       ## List of files sorted by name
    content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"],engine = 'netcdf4',chunks={"time": 240})
    xr_aff_pft.append(content)


scenario = 'ssp370'
xr_ctl_pft = []
for i,mm in enumerate(models):
    filepath = glob.glob(os.path.join(data_path + 'xr_' + scenario + "_" + mm + '_pft.nc'))[0]                                       ## List of files sorted by name
    content = xr.open_dataset(filepath, drop_variables=["time_bnds","lon_bnds","lat_bnds"],engine = 'netcdf4',chunks={"time": 240})
    xr_ctl_pft.append(content)


Preprocessing

In [15]:
# Convert pr and evspsbl from kg m-2 s-1 to mm month-1
for i,mm in enumerate(models):
    xr_ctl[i]["pr"] = xr_ctl[i].pr*86400* xr_ctl[i].time.dt.days_in_month
    xr_aff[i]["pr"] = xr_aff[i].pr*86400* xr_aff[i].time.dt.days_in_month
    
    xr_ctl[i]["pr"].attrs=dict(units="mm/months")
    xr_aff[i]["pr"].attrs=dict(units="mm/months")

    xr_ctl[i]["evspsbl"] = xr_ctl[i].evspsbl*86400* xr_ctl[i].time.dt.days_in_month
    xr_aff[i]["evspsbl"] = xr_aff[i].evspsbl*86400* xr_aff[i].time.dt.days_in_month
    
    xr_ctl[i]["evspsbl"].attrs=dict(units="mm/months")
    xr_aff[i]["evspsbl"].attrs=dict(units="mm/months")

#### PET estimation according to Priestley-Taylor 

In [16]:
Cp = 1.0051*10**(-3)   # Specific heat of air at constant pressure (C_p)in MJ/(kg K)
Lv = 2.45              # Latent Heat of Vaporization for Water.  FAO uses constant 2.45 MJ/kg


In [17]:
def PriestleyTaylor(xr, alpha=1.26):
    # From https://www.nature.com/articles/s41597-023-02290-0
    '''Calculate Priestley-Taylor potential evapotranspiration for the given dictionary of input files'''
    
    # Convert to Celsius if temperature is in Kelvin
    # if xr['ts'].attrs['units'] == 'K':
    ts=xr['ts']-273.15

    # Net radiation = H + LE
    Rn = xr.hfss + xr.hfls
    
    # Psychometric constant
    psy = Cp*xr.ps/(1000*Lv*.622)                             # kPa
    
    # Saturation Vapour Pressure
    sat = (0.61078**(17.27*(ts)/(ts+237.3)))            # kPa
    
    # Delta
    delta = 4098*sat/((ts+237.3)**2)                     # kPa/K
    
    # Calculate PET using the Priestley-Taylor formulation with constant alpha (defalut alpha = 1.26)
    xr['pet'] = (alpha*delta*Rn)/(Lv*(delta+psy))             # mm/month

    # Mask out the ocean
    mask = np.ma.masked_equal(xr.lai, np.nan) 
    xr['pet'] = xr['pet'] * mask

    # Define variable attributes
    xr['pet'].attrs['units'] = 'mm month-1'
    xr['pet'].attrs['short_name'] = 'PET'
    xr['pet'].attrs['long_name'] = 'Priestley-Taylor Potential Evapotranspriation'
    xr['pet'].attrs['description'] = 'Priestley-Taylor Potential Evapotranspiration computed using formulation described in Priestley & Taylor (1972) with alpha = '+str(alpha)
    
    # # Aridity Index
    # xr['ai'] = xr['pet']*xr['pr']**-1
    # xr['ai'].attrs['units'] = 'mm month-1'
    # xr['ai'].attrs['short_name'] = 'AI'
    # xr['ai'].attrs['long_name'] = 'Aridity Index'
    # xr['ai'].attrs['description'] = 'PET/P'
      
    return xr

In [18]:
for i, mm in enumerate(models):
    PriestleyTaylor(xr_ctl[i], alpha=1.26)
    PriestleyTaylor(xr_aff[i], alpha=1.26) 

In [19]:
xr_delta = []
xr_delta_pft = []
for i,mm in enumerate(models):
    xr_delta.append(xr_aff[i].sel(time = slice("2071-01","2100-12")).mean(dim = "time") - xr_ctl[i].sel(time = slice("2071-01","2100-12")).mean(dim = "time"))
    xr_delta_pft.append(xr_aff_pft[i].sel(time = slice("2071-01","2100-12")).mean(dim = "time") - xr_ctl_pft[i].sel(time = slice("2071-01","2100-12")).mean(dim = "time"))

#### Water-Energy limitation

In [20]:
def regime_limit(dataset):
    # Definition of water-limited regions as from Forzieri et al., (2020) (https://www.nature.com/articles/s41558-020-0717-0)
    # Water-limited: cor(le,pr) > cor(le,ts) and vice-versa for Energy-limited regions
    cor_pr = lag_linregress_3D(dataset['hfls'], dataset['pr'])['cor'].compute()
    cor_ts = lag_linregress_3D(dataset['hfls'], dataset['ts'])['cor'].compute()
    
    water_limited = dataset.isel(time = -1).where(cor_pr > cor_ts, drop=True)["lai"]
    energy_limited = dataset.isel(time = -1).where(cor_pr < cor_ts, drop=True)["lai"]
    
    # Ensure computation and time slice selection
    return water_limited, energy_limited

In [21]:
mask_aff_water = []
mask_aff_energy = []
mask_ctl_water = []
mask_ctl_energy = []
for i, mm in enumerate(models):
    water, energy = regime_limit(xr_aff[i])
    mask_aff_water.append(water)
    mask_aff_energy.append(energy)
    
    water, energy = regime_limit(xr_ctl[i])
    mask_ctl_water.append(water)
    mask_ctl_energy.append(energy)

### Mask treeFrac

In [22]:
%%capture

dtree = []

# masking regions of treefrac increase or decrease
mask_treefrac_pos = []
mask_treefrac_neg = []

for m,_ in enumerate(models):
    
    # identify regions with 5%+ and 5%- treefrac
    mask_treefrac_pos.append(xr_delta_pft[m]["treeFrac"].where(xr_delta_pft[m]["treeFrac"] > 10))
    mask_treefrac_neg.append(xr_delta_pft[m]["treeFrac"].where(xr_delta_pft[m]["treeFrac"] <- 10))


### Effect on PET

In [ ]:
%%capture
regions = ["NA", "AFR", "SE Asia", "SA", "EUR"]

boxes = [
        (-130, -60, 30, 60),    # NA
        (-20, 50, -25, 20),     # Africa
        (90, 130, 10, 40),       # SEAsia
        (-85, -35, -40, 10),       # SA
        (10, 60, 40, 70)       # Eurasia
]

df_delta_pos = []
df_delta_neg = []
for i,mm in enumerate(models):
    content = {}
    content1 = {}
    for bb,b in enumerate(boxes):
        content[regions[bb]] = ((xr.merge([xr_delta[i],xr_delta_pft[i]]).sel(lon = slice(b[0], b[1]), lat = slice(b[2], b[3]))
                              .where(mask_treefrac_pos[i].notnull())
                              .stack(cell=["lon", "lat"])).compute().dropna(dim='cell')).to_dataframe().drop(columns = ["lon", "lat","height"])
        content1[regions[bb]] = ((xr.merge([xr_delta[i],xr_delta_pft[i]]).sel(lon = slice(b[0], b[1]), lat = slice(b[2], b[3]))
                              .where(mask_treefrac_neg[i].notnull())
                              .stack(cell=["lon", "lat"])).compute().dropna(dim='cell')).to_dataframe().drop(columns = ["lon", "lat","height"])
    
    df_delta_pos.append(content)
    df_delta_neg.append(content1)

In [ ]:
indices = [0,1,3]
regions_pos = [regions[i] for i in indices]

width_inch = 14
height_inch = 6

fig = plt.figure(figsize=(width_inch, height_inch)) #, constrained_layout=True)
gs = gridspec.GridSpec(3, 5)

cmap = "RdBu"
norm = colors.Normalize(vmin=-3, vmax=3)

for m, mm in enumerate(models):
    for r, rr in enumerate(regions_pos):
        data=df_delta_pos[m][rr]
        
        axs = fig.add_subplot(gs[r,m])
        scatter = axs.scatter(
                             x = data["treeFrac"],
                             y = data["hfss"],
                             c = data["pet"],
                             cmap = cmap
                            
        )
        # axs.set_ylim(-1*10**-5, 1*10**-5)
        axs.set_xlim(8, 80)
        # axs.legend(loc='upper right', fontsize = 10)
        axs.set_ylabel("")
        if m == 0:
            axs.set_ylabel("pet"+ " - " + rr)
        if r == 2:
            axs.set_xlabel("treeFrac")
        if r == 0:
            axs.set_title(models[m])
        # axs.label_outer()
        
        axs.set_xticklabels("")
# fig.tight_layout()
cbar_ax = fig.add_axes([0.82, 0.3, 0.02, 0.4])  # Adjust these values to position your colorbar
cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax)
cbar.set_label("Latent Heat (Wm${-2}$)")
fig.subplots_adjust(bottom=0.2, top=0.95, left=0.1, right=0.8, wspace=0.2, hspace=0.25)
fig.suptitle("pet - positive treeFrac >10%", y =1.05, x = 0.45)

### Account for Water-Energy limitation 

In [ ]:
%%capture
regions = ["NA", "AFR", "SE Asia", "SA", "EUR"]

boxes = [
        (-130, -60, 30, 60),    # NA
        (-20, 50, -25, 20),     # Africa
        (90, 130, 10, 40),       # SEAsia
        (-85, -35, -40, 10),       # SA
        (10, 60, 40, 70)       # Eurasia
]

df_delta_pos_water = []
df_delta_pos_energy = []
for i,mm in enumerate(models):
    content = {}
    content1 = {}
    for bb,b in enumerate(boxes):
        content[regions[bb]] = ((xr.merge([xr_delta[i],xr_delta_pft[i]]).sel(lon = slice(b[0], b[1]), lat = slice(b[2], b[3]))
                              .where(mask_treefrac_pos[i].notnull()).where(mask_aff_water[i].notnull())
                              .stack(cell=["lon", "lat"])).compute().dropna(dim='cell')).to_dataframe().drop(columns = ["lon", "lat","height"])
        content1[regions[bb]] = ((xr.merge([xr_delta[i],xr_delta_pft[i]]).sel(lon = slice(b[0], b[1]), lat = slice(b[2], b[3]))
                              .where(mask_treefrac_pos[i].notnull()).where(mask_aff_energy[i].notnull())
                              .stack(cell=["lon", "lat"])).compute().dropna(dim='cell')).to_dataframe().drop(columns = ["lon", "lat","height"])
    
    df_delta_pos_water.append(content)
    df_delta_pos_energy.append(content1)

In [ ]:
df_delta_pos_water[4]["AFR"].columns #.mean(axis = 0)

In [ ]:
indices = [1,3]
regions_pos = [regions[i] for i in indices]

width_inch = 14
height_inch = 6

fig = plt.figure(figsize=(width_inch, height_inch)) #, constrained_layout=True)
gs = gridspec.GridSpec(2, 5)

cmap = "RdBu"
norm = colors.Normalize(vmin=-3, vmax=3)

for m, mm in enumerate(models):
    for r, rr in enumerate(regions_pos):
        data=df_delta_pos_water[m][rr]
        
        axs = fig.add_subplot(gs[r,m])
        scatter = axs.scatter(
                             x = data["treeFrac"],
                             y = data["hfss"],
                             c = data["pet"],
                             cmap = cmap
                            
        )
        # axs.set_ylim(-1*10**-5, 1*10**-5)
        # axs.set_xlim(8, 80)
        # axs.legend(loc='upper right', fontsize = 10)
        axs.set_ylabel("")
        if m == 0:
            axs.set_ylabel("pet"+ " - " + rr)
        if r == 2:
            axs.set_xlabel("treeFrac")
        if r == 0:
            axs.set_title(models[m])
            axs.set_xticklabels("")
        # axs.label_outer()
        

# fig.tight_layout()
cbar_ax = fig.add_axes([0.82, 0.3, 0.02, 0.4])  # Adjust these values to position your colorbar
cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax)
cbar.set_label("Latent Heat (Wm${-2}$)")
fig.subplots_adjust(bottom=0.2, top=0.95, left=0.1, right=0.8, wspace=0.2, hspace=0.25)
fig.suptitle("Water limited, treeFrac >10%", y =1.05, x = 0.45)

In [ ]:
indices = [0,1,3]
regions_pos = [regions[i] for i in indices]

width_inch = 14
height_inch = 8

fig = plt.figure(figsize=(width_inch, height_inch)) #, constrained_layout=True)
gs = gridspec.GridSpec(3, 5)

cmap = "RdBu"
norm = colors.Normalize(vmin=-3, vmax=3)

for m, mm in enumerate(models):
    for r, rr in enumerate(regions_pos):
        data=df_delta_pos_energy[m][rr]
        
        axs = fig.add_subplot(gs[r,m])
        scatter = axs.scatter(
                             x = data["treeFrac"],
                             y = data["pet"],
                             c = data["hfls"],
                             cmap = cmap
                            
        )
        # axs.set_ylim(-1*10**-5, 1*10**-5)
        # axs.set_xlim(8, 80)
        # axs.legend(loc='upper right', fontsize = 10)
        axs.set_ylabel("")
        if m == 0:
            axs.set_ylabel("pet"+ " - " + rr)
        if r == 2:
            axs.set_xlabel("treeFrac")
        if r == 0:
            axs.set_title(models[m])
            axs.set_xticklabels("")
        # axs.label_outer()
        

# fig.tight_layout()
cbar_ax = fig.add_axes([0.82, 0.3, 0.02, 0.4])  # Adjust these values to position your colorbar
cbar = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax)
cbar.set_label("Latent Heat (Wm${-2}$)")
fig.subplots_adjust(bottom=0.2, top=0.95, left=0.1, right=0.8, wspace=0.2, hspace=0.25)
fig.suptitle("Energy limited, treeFrac >10%", y =1.05, x = 0.45)

## Multilinear Regression

### Regression with {locator}

In [23]:
def detrend_dim(da, dim, degree):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=degree)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    da_det = (da - fit)
    return da_det

def pval_calc(x, y, ypred):
    n = X.shape[0]
    p = X.shape[1] 

    # Calculate residuals
    res = y - ypred

    # Calculate the standard error of the regression
    RSS = np.sum(res**2)
    MSE = RSS / (n - p - 1)
    # X_with_intercept = np.column_stack((np.ones(n), X))
    X_with_intercept = np.append(np.ones((len(X),1)), X, axis=1)

    var_b = MSE * np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y

    # Standard errors of the coefficients
    std_b = np.sqrt(var_b)

    # t-statistics for the coefficients
    t_stats = coef / std_b[1:]  # Exclude intercept's standard error

    # Two-tailed p-values for the t-statistics
    p_values = [2 * (1 - stats.t.cdf(np.abs(t), df=n - p - 1)) for t in t_stats]
    p_values = np.append(np.nan, p_values)[1:]  # The intercept doesn't have a p-value                                                   
    return p_values

In [24]:
%%capture

from sklearn import linear_model
from regressors import stats

linear = linear_model.LinearRegression()
# linear = linear_model.Ridge()

coef_reg = []
pval_reg = []
r2_reg = []
ypred_reg = []

# Define the list of predictors
variables = ["lai","ts","pr","rsds"]

for m, mm in enumerate(models):
    data = (xr_aff[m] - xr_ctl[m])
    data_pft = (xr_aff_pft[m] - xr_ctl_pft[m])
    
    ## Preprocessing
    # Dimension time on 0th function argument to apply_ufunc with dask='parallelized' consists of multiple chunks, but is also a core dimension. 
    # To fix, either rechunk into a single dask array chunk along this dimension, i.e., ``.compute()`
    # Replace NaN and 0 with 0.01 to avoid Singular Matrix 

    # X1 = data.lai.resample(time='Y', label='left').mean().compute()

    X1 = data_pft.treeFrac.resample(time='Y', label='left').mean().compute()
    X2 = data.ts.resample(time='Y', label='left').mean().compute()
    X3 = data.pr.resample(time='Y', label='left').sum().compute()
    X4 = data.rsds.resample(time='Y', label='left').mean().compute()
    Y = data.hfls.resample(time='Y', label='left').mean().compute()
    
    # 1st order detrending
    X1 = detrend_dim(X1, "time", 1)
    X2 = detrend_dim(X2, "time", 1)
    X3 = detrend_dim(X3, "time", 1)
    X4 = detrend_dim(X4, "time", 1)
    Y = detrend_dim(Y, "time", 1)
    
    ## Standardization
    X1 = (X1 - X1.mean(dim = ["time"]))/X1.std(dim = ["time"])
    X2 = (X2 - X2.mean(dim = ["time"]))/X2.std(dim = ["time"])
    X3 = (X3 - X3.mean(dim = ["time"]))/X3.std(dim = ["time"])
    X4 = (X4 - X4.mean(dim = ["time"]))/X4.std(dim = ["time"])
    
    # Gap filling
    X1 = X1.fillna(0.001); X1 = X1.where(X1 != 0, 0.001)
    X2 = X2.fillna(0.001); X2 = X2.where(X2 != 0, 0.001)
    X3 = X3.fillna(0.001); X3 = X3.where(X3 != 0, 0.001)
    X4 = X4.fillna(0.001); X4 = X4.where(X4 != 0, 0.001)
    Y = Y.fillna(0.001); Y = Y.where(Y != 0, 0.001)

    # Stack on 1D vectors
    X1 = X1.stack(cell = ["lon","lat"]); X2 = X2.stack(cell = ["lon","lat"]); X3 = X3.stack(cell = ["lon","lat"]); X4 = X4.stack(cell = ["lon","lat"]); Y = Y.stack(cell = ["lon","lat"])

    # Create empty dataarray to store regression diagnostics
    xr_coef = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
    xr_pval = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
    xr_r2 = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
    xr_ypred = xr.DataArray(data=None, coords=[X1.time,X1.cell], dims=["time","cell"])
    for i,vars in enumerate(variables):
        xr_coef[vars] = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
        xr_pval[vars] = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])

        
    # Iterate over cells
    for c in X1.cell:
        locator = {'cell':c}
        
        X = np.array((X1.loc[locator].values, X2.loc[locator].values, X3.loc[locator].values, X4.loc[locator].values)); X = X.T

        # Check if all elements in X are 0.001
        # if np.all(X == 0.001):
        # Store regression diagnostics
        # for i,vars in enumerate(["lai","ts","pr","rsds"]):
        #     xr_coef[vars].loc[locator] = xr.DataArray(data = np.nan); 
        #     xr_pval[vars].loc[locator] = xr.DataArray(data = np.nan); 
        # xr_r2.loc[locator] = xr.DataArray(data = np.nan); 
        # xr_ypred.loc[locator] =  xr.DataArray(data = np.nan)            
        #     continue  # Skip this iteration if all elements are 0.001
            
        # Regression
        model = linear.fit(X, Y.loc[locator].values)
        r2 = linear.score(X, Y.loc[locator].values)
        ypred = linear.predict(X)
        coef = model.coef_
        # p_values = pval_calc(X, Y.loc[locator].values, ypred)
        p_values = stats.coef_pval(model,X,Y.loc[locator].values)[1:]
    
        # Store regression diagnostics
        for i,vars in enumerate(variables):
            xr_coef[vars].loc[locator] = xr.DataArray(data = coef[i]); 
            xr_pval[vars].loc[locator] = xr.DataArray(data = p_values[i]); 
        xr_r2.loc[locator] = xr.DataArray(data = r2);
        xr_ypred.loc[locator] =  xr.DataArray(data = ypred)
     
    xr_coef = xr_coef.unstack()
    xr_pval = xr_pval.unstack()
    xr_r2 = xr_r2.unstack()
    xr_ypred = xr_ypred.unstack()
    
    # Convert object results to np.float64
    for i,vars in enumerate(variables):
        xr_coef[vars] = xr_coef[vars].astype(np.float64)
        xr_pval[vars] = xr_pval[vars].astype(np.float64)
    xr_r2 = xr_r2.astype(np.float64)
    xr_ypred = xr_ypred.astype(np.float64)
    
    coef_reg.append(xr_coef)
    pval_reg.append(xr_pval)
    r2_reg.append(xr_r2)
    ypred_reg.append(xr_ypred)
    
# Save and export regression list data
import pickle
data_path = '/home/m/m301093/data/'

with open(os.path.join(data_path+"ols_reg_coef_LE_treeFrac"), "wb") as fp:   #Pickling
    pickle.dump(coef_reg, fp)

with open(os.path.join(data_path+"ols_reg_pval_LE_treeFrac"), "wb") as fp:   #Pickling
    pickle.dump(pval_reg, fp)

with open(os.path.join(data_path+"ols_reg_r2_LE_treeFrac"), "wb") as fp:   #Pickling
    pickle.dump(r2_reg, fp)

with open(os.path.join(data_path+"ols_reg_ypred_LE_treeFrac"), "wb") as fp:   #Pickling
    pickle.dump(ypred_reg, fp)


### Regression with {wrapper}

In [1]:
def detrend_dim(da, dim, degree):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=degree)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    da_det = (da - fit)
    return da_det

In [35]:
%%capture

from sklearn import linear_model
from regressors import stats

linear = linear_model.LinearRegression()
# linear = linear_model.Ridge()

def xr_MLR(x1, x2, x3, x4, y):
      
    # Wrapper around scipy linregress to use in apply_ufunc
    x1 = x1.reshape(-1,1); x2 = x2.reshape(-1,1); x3 = x3.reshape(-1,1); x4 = x4.reshape(-1,1) #; x5 = x5.reshape(-1,1); x6 = x6.reshape(-1,1)#; x7 = x7.reshape(-1,1)
    y = y.reshape(-1,1)
    X = np.concatenate((x1, x2, x3, x4), axis = 1)
    model = linear.fit(X, y)
    r2 = linear.score(X, y)
    ypred = linear.predict(X)
    coef = model.coef_
    pval = stats.coef_pval(model,X,y)[1:] 

    # pval = stats.coef_pval(model,X,y)[1:]         # First p-value [0] is for intercept
    return coef,pval,np.array([r2])

coef_reg = []
pval_reg = []
r2_reg = []
for m, mm in enumerate(models):
    data = (xr_aff[m] - xr_ctl[m])
    data_pft = (xr_aff_pft[m] - xr_ctl_pft[m])
    
    ## Preprocessing
    # Dimension time on 0th function argument to apply_ufunc with dask='parallelized' consists of multiple chunks, but is also a core dimension. 
    # To fix, either rechunk into a single dask array chunk along this dimension, i.e., ``.compute()`
    # Replace NaN and 0 with 0.01 to avoid Singular Matrix 
    X1 = data.lai.resample(time='Y', label='left').mean().compute(); 
    X2 = data.ts.resample(time='Y', label='left').mean().compute(); 
    X3 = data.pr.resample(time='Y', label='left').sum().compute(); 
    X4 = data.rsds.resample(time='Y', label='left').mean().compute(); 
    Y = data.hfls.resample(time='Y', label='left').mean().compute(); 

    ## Standardization
    X1 = (X1 - X1.mean(dim = ["time"]))/X1.std(dim = ["time"])
    X2 = (X2 - X2.mean(dim = ["time"]))/X2.std(dim = ["time"])
    X3 = (X3 - X3.mean(dim = ["time"]))/X3.std(dim = ["time"])
    X4 = (X4 - X4.mean(dim = ["time"]))/X4.std(dim = ["time"])
    
    # 1st order detrending
    X1 = detrend_dim(X1, "time", 1)
    X2 = detrend_dim(X2, "time", 1)
    X3 = detrend_dim(X3, "time", 1)
    X4 = detrend_dim(X4, "time", 1)
    Y = detrend_dim(Y, "time", 1)

    X1 = X1.fillna(0.001) #.where(X1 != 0, 0.001)
    X2 = X2.fillna(0.001) #.where(X2 != 0, 0.001)
    X3 = X3.fillna(0.001) #.where(X3 != 0, 0.001)
    X4 = X4.fillna(0.001) #.where(X4 != 0, 0.001)
    Y = Y.fillna(0.001) #.where(Y != 0, 0.001)
    
    # return new DataArrays
    xr_coef, xr_pval, xr_r2 = xr.apply_ufunc(xr_MLR,
                                            X1, X2, X3, X4, Y,
                                            input_core_dims=[['time'],['time'], ['time'], ['time'],['time']],
                                            output_core_dims=[['coef'], ['pval'], ['r2']],
                                            vectorize=True,
                                            dask="parallelized",
                                            # allow_rechunk="True",
                                            output_dtypes=['float64', 'float64','float64'],
                                            output_sizes={"coef":4, "pval":4, "r2":1},
                                   )
    coef_reg.append(xr_coef)
    pval_reg.append(xr_pval)
    r2_reg.append(xr_r2)

# Save and export regression list data
import pickle
data_path = '/home/m/m301093/data/'

with open(os.path.join(data_path+"ols_reg_coef_wrap"), "wb") as fp:   #Pickling
    pickle.dump(coef_reg, fp)

with open(os.path.join(data_path+"ols_reg_pval_wrap"), "wb") as fp:   #Pickling
    pickle.dump(pval_reg, fp)

with open(os.path.join(data_path+"ols_reg_r2_wrap"), "wb") as fp:   #Pickling
    pickle.dump(r2_reg, fp)

with open(os.path.join(data_path+"ols_reg_ypred_wrap"), "wb") as fp:   #Pickling
    pickle.dump(ypred_reg, fp)

In [52]:
X1.to_numpy().reshape(-1,1)
X = np.concatenate((X1.to_numpy().reshape(-1,1), X2.to_numpy().reshape(-1,1), X3.to_numpy().reshape(-1,1), X4.to_numpy().reshape(-1,1)), axis = 1)

In [61]:
mask = np.all(X == 0.001, axis=1)
filtered_data = X[~mask]

In [62]:
filtered_data.shape

(1473696, 4)

In [46]:
# Check if any variable in the Dataset contains NaNs
has_nan = X4.isnull().any().item()

print(f"Dataset contains NaN: {has_nan}")

Dataset contains NaN: False


### reg on pair of consecutive years

In [ ]:
# %%capture

from sklearn import linear_model
from regressors import stats

linear = linear_model.LinearRegression()
# linear = linear_model.Ridge()

def xr_MLR(x1, x2, x3, x4, y):
      
    # Wrapper around scipy linregress to use in apply_ufunc
    x1 = x1.reshape(-1,1); x2 = x2.reshape(-1,1); x3 = x3.reshape(-1,1); x4 = x4.reshape(-1,1) #; x5 = x5.reshape(-1,1); x6 = x6.reshape(-1,1)#; x7 = x7.reshape(-1,1)
    y = y.reshape(-1,1)
    X = np.concatenate((x1, x2, x3, x4), axis = 1)
    model = linear.fit(X, y)
    r2 = linear.score(X, y)
    ypred = linear.predict(X)
    coef = model.coef_
    pval = stats.coef_pval(model,X,y)[1:] 

    # pval = stats.coef_pval(model,X,y)[1:]         # First p-value [0] is for intercept
    return coef,pval,np.array([r2])

coef_reg = []
pval_reg = []
r2_reg = []
for m, mm in enumerate(models):
    data = (xr_aff[m] - xr_ctl[m])
    data_pft = (xr_aff_pft[m] - xr_ctl_pft[m])
    
    # Dimension time on 0th function argument to apply_ufunc with dask='parallelized' consists of multiple chunks, but is also a core dimension. 
    # To fix, either rechunk into a single dask array chunk along this dimension, i.e., ``.compute()`
    # Replace NaN and 0 with 0.01 to avoid Singular Matrix 
    X1 = data.lai.resample(time='Y', label='left').mean().compute()
    X2 = data.ts.resample(time='Y', label='left').mean().compute()
    X3 = data.pr.resample(time='Y', label='left').sum().compute()
    X4 = data.rsds.resample(time='Y', label='left').mean().compute()
    Y = data.hfls.resample(time='Y', label='left').mean().compute()
    
    X1 = X1.fillna(0.001); X1 = X1.where(X1 != 0, 0.001)
    X2 = X2.fillna(0.001); X2 = X2.where(X2 != 0, 0.001)
    X3 = X3.fillna(0.001); X3 = X3.where(X3 != 0, 0.001)
    X4 = X4.fillna(0.001); X4 = X4.where(X4 != 0, 0.001)
    Y = Y.fillna(0.001); Y = Y.where(Y != 0, 0.001)
    ## Preprocessing
    # Standardization
    X1 = X1 - X1.mean(dim = ["time"])/X1.std(dim = ["time"])
    X2 = X2 - X2.mean(dim = ["time"])/X2.std(dim = ["time"])
    X3 = X3 - X3.mean(dim = ["time"])/X3.std(dim = ["time"])
    X4 = X4 - X4.mean(dim = ["time"])/X4.std(dim = ["time"])

    
    # Compute the regression for every pair of consecutive years
    years = np.arange(0,X1.time.shape[0]-1,1)
    coef_m = []; pval_m = []; r2_m = []
    for t in years:
        X1 = X1.isel(time = [t,t+1]); X2 = X2.isel(time = [t,t+1]); X3 = X3.isel(time = [t,t+1]); X4 = X4.isel(time = [t,t+1])
        Y = Y.isel(time = [t,t+1])
        # return new DataArrays
        coef, pval, r2 = xr.apply_ufunc(xr_MLR,
                                                X1, X2, X3, X4, Y,
                                                input_core_dims=[['time'],['time'], ['time'], ['time'],['time']],
                                                output_core_dims=[['coef'], ['pval'], ['r2']],
                                                vectorize=True,
                                                dask="parallelized",
                                                # allow_rechunk="True",
                                                output_dtypes=['float64', 'float64', 'float64'],
                                                output_sizes={"coef":4, "pval":4, "r2":4},
                                       )
        coef_m.append(coef); pval_m.append(pval); r2_m.append(r2)
    coef_m = xr.concat(coef_m,dim = "delta").mean(dim = "delta")
    pval_m = xr.concat(pval_m,dim = "delta").mean(dim = "delta")
    r2_m = xr.concat(r2_m,dim = "delta").mean(dim = "delta")
    
    coef_reg.append(coef)
    pval_reg.append(pval)
    r2_reg.append(r2)

In [ ]:
from sklearn import linear_model
from regressors import stats

linear = linear_model.LinearRegression()
# linear = linear_model.Ridge()

coef_reg = []
pval_reg = []
r2_reg = []

## For every ESMs
for m, mm in enumerate(models):
    data = (xr_aff[m] - xr_ctl[m])
    data_pft = (xr_aff_pft[m] - xr_ctl_pft[m])
    
    # Dimension time on 0th function argument to apply_ufunc with dask='parallelized' consists of multiple chunks, but is also a core dimension. 
    # To fix, either rechunk into a single dask array chunk along this dimension, i.e., ``.compute()`
    # Replace NaN and 0 with 0.01 to avoid Singular Matrix 
    X1 = data.lai.resample(time='Y', label='left').mean().compute()
    X2 = data.ts.resample(time='Y', label='left').mean().compute()
    X3 = data.pr.resample(time='Y', label='left').sum().compute()
    X4 = data.rsds.resample(time='Y', label='left').mean().compute()
    Y = data.hfls.resample(time='Y', label='left').mean().compute()
    
    X1 = X1.fillna(0.001); X1 = X1.where(X1 != 0, 0.001)
    X2 = X2.fillna(0.001); X2 = X2.where(X2 != 0, 0.001)
    X3 = X3.fillna(0.001); X3 = X3.where(X3 != 0, 0.001)
    X4 = X4.fillna(0.001); X4 = X4.where(X4 != 0, 0.001)
    Y = Y.fillna(0.001); Y = Y.where(Y != 0, 0.001)
    ## Preprocessing
    # Standardization
    X1 = X1 - X1.mean(dim = ["time"])/X1.std(dim = ["time"])
    X2 = X2 - X2.mean(dim = ["time"])/X2.std(dim = ["time"])
    X3 = X3 - X3.mean(dim = ["time"])/X3.std(dim = ["time"])
    X4 = X4 - X4.mean(dim = ["time"])/X4.std(dim = ["time"])

    # Gap filling
    X1 = X1.fillna(0.001); X1 = X1.where(X1 != 0, 0.001)
    X2 = X2.fillna(0.001); X2 = X2.where(X2 != 0, 0.001)
    X3 = X3.fillna(0.001); X3 = X3.where(X3 != 0, 0.001)
    X4 = X4.fillna(0.001); X4 = X4.where(X4 != 0, 0.001)
    Y = Y.fillna(0.001); Y = Y.where(Y != 0, 0.001)

    # Stack on 1D vectors
    X1 = X1.stack(cell = ["lon","lat"]); X2 = X2.stack(cell = ["lon","lat"]); X3 = X3.stack(cell = ["lon","lat"]); X4 = X4.stack(cell = ["lon","lat"]); Y = Y.stack(cell = ["lon","lat"])

    # Create empty dataarray to store regression diagnostics
    xr_coef = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
    xr_pval = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
    xr_r2 = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
    xr_ypred = xr.DataArray(data=None, coords=[X1.time,X1.cell], dims=["time","cell"])
    for i,vars in enumerate(variables):
        xr_coef[vars] = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])
        xr_pval[vars] = xr.DataArray(data=None, coords=[X1.cell], dims=["cell"])

    # Compute the regression for every pair of consecutive years
    years = np.arange(0,X1.time.shape[0]-1,1)
    coef_m = []; pval_m = []; r2_m = []
    for t in years:
        X1 = X1.isel(time = [t,t+1]); X2 = X2.isel(time = [t,t+1]); X3 = X3.isel(time = [t,t+1]); X4 = X4.isel(time = [t,t+1])
        Y = Y.isel(time = [t,t+1])

        for c in X1.cell:
            locator = {'cell':c}
            
            X = np.array((X1.loc[locator].values, X2.loc[locator].values, X3.loc[locator].values, X4.loc[locator].values)); X = X.T

            # Regression
            model = linear.fit(X, Y.loc[locator].values)
            r2 = linear.score(X, Y.loc[locator].values)
            ypred = linear.predict(X)
            coef = model.coef_
            # p_values = pval_calc(X, Y.loc[locator].values, ypred)
            p_values = stats.coef_pval(model,X,Y.loc[locator].values)[1:]
        
            # Store regression diagnostics
            for i,vars in enumerate(variables):
                xr_coef[vars].loc[locator] = xr.DataArray(data = coef[i]); 
                xr_pval[vars].loc[locator] = xr.DataArray(data = p_values[i]); 
            xr_r2.loc[locator] = xr.DataArray(data = r2);
            xr_ypred.loc[locator] =  xr.DataArray(data = ypred)
     
        xr_coef = xr_coef.unstack()
        xr_pval = xr_pval.unstack()
        xr_r2 = xr_r2.unstack()
        xr_ypred = xr_ypred.unstack()
    
    # Convert object results to np.float64
    for i,vars in enumerate(variables):
        xr_coef[vars] = xr_coef[vars].astype(np.float64)
        xr_pval[vars] = xr_pval[vars].astype(np.float64)
    xr_r2 = xr_r2.astype(np.float64)
    xr_ypred = xr_ypred.astype(np.float64)
        coef_m.append(coef); pval_m.append(pval); r2_m.append(r2)
    coef_m = xr.concat(coef_m,dim = "delta").mean(dim = "delta")
    pval_m = xr.concat(pval_m,dim = "delta").mean(dim = "delta")
    r2_m = xr.concat(r2_m,dim = "delta").mean(dim = "delta")
    
    coef_reg.append(coef)
    pval_reg.append(pval)
    r2_reg.append(r2)